In [1]:
'''
make_submission() generates predictions for the Kaggle Painter by Numbers competion
using simple features (image size, aspect ratio and bits/pixel^2)
author: Swaroop Krothapalli - extended code of small yello duck
https://github.com/swaroop7/painters
'''
import time
import os
import random
import numpy as np
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
from sklearn.cross_validation import KFold
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score  
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, auc
from scipy.stats import itemfreq
from sklearn import neighbors, linear_model
from sklearn.metrics import classification_report
import xgboost as xgb

mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-5.3.0-posix-seh-rt_v4-rev0\\mingw64\\bin'
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

os.chdir('C:\\Users\\swaroop\\Downloads\\painters\\ImageSets\\Wyeths')

def getEntropy(signal):
    lensig=signal.size
    symset=list(set(signal))
    numsym=len(symset)
    probabability_distribution=[np.size(signal[signal==i])/(1.0*lensig) for i in symset]
    entropy=np.sum([p*np.log2(1.0/p) for p in probabability_distribution])
    return entropy

def calculateEntropyNeighbourhood(artwork, neighbourhood):
    image = cv2.imread(artwork)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    colorIm=np.array(image)
    grayIm=np.array(gray_image)
    
    N=neighbourhood
    S=grayIm.shape
    E=np.array(grayIm)
    
    for row in range(S[0]): 
            for col in range (S[1]): 
                    Lx=np.max([0,col-N]) 
                    Ux=np.min([S[1],col+N])
                    Ly=np.max([0,row-N])
                    Uy=np.min([S[0],row+N])
                    # makes region 1-D
                    region=grayIm[Ly:Uy,Lx:Ux].flatten()
                    E[row,col]=getEntropy(region)
    
    average=np.mean(E)
    return average

def getDTM(artwork):
    image = cv2.imread(artwork)
    image32f = np.float32(image)
    mu    = cv2.blur(image32f,(5,5))
    mu2   = cv2.blur(cv2.multiply(image32f,image32f), (5,5))
    sigma = cv2.sqrt( mu2 - cv2.multiply(mu, mu) )
    return np.mean(sigma)

#image_info_test = get_image_info(test_info, 'test')
def get_image_info(test_info, dir):
	if dir == 'test':
		images = list(set(list(test_info.image1.unique()) + list(test_info.image2.unique())))
		info = pd.DataFrame(np.array(images).reshape((-1, 1)), columns = ['filename'])
		#print info
	else:
		info = test_info
	
	info['pixelsx'] = np.nan
	info['pixelsy'] = np.nan
	info['size_bytes'] = np.nan
# 	info['entropy1'] = np.nan
# 	info['entropy5'] = np.nan
# 	info['entropy10'] = np.nan
# 	info['entropy15'] = np.nan
# 	info['entropy20'] = np.nan
	info['dtm'] = np.nan

    
	
	j = 0
	for i in info.index.values:
		j += 1        
		try:
			#print i
			#fil = 'C:\\Users\\swaroop\\Downloads\\painters\\ImageSets\\Wyeths\\'+dir+'\\'+info.loc[i, 'filename']
			fil = 'C:\\Users\\swaroop\\Downloads\\painters\\ImageSets\\Wyeths\\'+info.loc[i, 'filename']
			#print fil
			im = Image.open('C:\\Users\\swaroop\\Downloads\\painters\\ImageSets\\Wyeths\\'+info.loc[i, 'filename'])
			#print im
			#print im.size
			info.loc[i, 'pixelsx'], info.loc[i, 'pixelsy'] = im.size
			#im = cv2.imread(dir+'/'+info.loc[i, 'new_filename'])
			#info.loc[i, 'pixelsx'], info.loc[i, 'pixelsy'] = im.shape[0:2]
			info.loc[i, 'size_bytes'] = os.path.getsize(info.loc[i, 'filename'])
			#info.loc[i, 'entropy'] = calculateEntropyNeighbourhood(fil, 1)
			#print calculateEntropyNeighbourhood(fil, 1)
			info.loc[i, 'dtm'] = getDTM(fil)
# 			info['entropy1'] = calculateEntropyNeighbourhood(fil, 1)
# 			info['entropy5'] = calculateEntropyNeighbourhood(fil, 5)
# 			info['entropy10'] = calculateEntropyNeighbourhood(fil, 10)
# 			info['entropy15'] = calculateEntropyNeighbourhood(fil, 15)
# 			info['entropy20'] = calculateEntropyNeighbourhood(fil, 20)

		except:
			print dir+'\\'+info.loc[i, 'filename']
		if (j%10 == 0):
			print '',
	info=info.dropna()
	print 'info shape',info.shape
	return info.rename(columns={'filename' : 'new_filename'})

#t = make_pairs(train_info)	
def make_pairs(train_info):
	print "make pairs train info shape",train_info.shape
	artists = train_info.artist.unique()

	n = train_info.groupby('artist').size()
	n = (2*n**2).sum() 
	t = pd.DataFrame(np.zeros((n, 4)), columns=['artist1', 'image1', 'artist2', 'image2'])
	i = 0
	j = 0
	for m in artists:
		
		a = train_info[train_info.artist==m][['artist', 'new_filename']].values
		use = train_info[train_info.artist != m].index.values
		print "a and use shapes", a.shape, use.shape
		np.random.shuffle(use)
		#print a.shape, use.shape
		nm = np.mean([a.shape[0]**2, train_info[train_info.artist != m].shape[0] ])
		print nm
		use = use[0:nm]
		print "use.shape",use.shape
		b = train_info[train_info.artist!=m][['artist', 'new_filename']].ix[use, :].values
		#print nm, use.shape, b.shape
		a2 = pd.DataFrame(np.concatenate([np.repeat(a[:, 0], a.shape[0]).reshape((-1,1)), np.repeat(a[:, 1], a.shape[0]).reshape((-1,1)), np.tile(a, (a.shape[0], 1))], axis=1), columns=['artist1', 'image1', 'artist2', 'image2'])
		a2 = a2.loc[0:nm, :]
		b2 = pd.DataFrame(np.concatenate([np.tile(a, (a.shape[0], 1))[0:b.shape[0], :], b], axis=1), columns=['artist1', 'image1', 'artist2', 'image2'])
		print j, i, a2.shape[0], b2.shape[0]
		#print b2
		t.iloc[i:i+a2.shape[0], :] = a2.values
		t.iloc[i+a2.shape[0]:i+a2.shape[0]+b2.shape[0], :] = b2.values
		i += a2.shape[0] +b2.shape[0]
		j += 1
	
	t = t[~t.image2.isin([np.nan, 0])]
	print t.shape, t[t.image1 > t.image2].shape
	print t.columns.values
	#print t
	print "hi1",t.drop_duplicates(subset=['artist1', 'artist2','image1', 'image2'], keep=False).shape
	#return t[t.image1 > t.image2]	
	return t.drop_duplicates(subset=['artist1', 'artist2','image1', 'image2'], keep=False)


#x_train, y_train, x_cv, y_cv = prep_data([train_info, None], 'cv')	
#x_test, y_test = prep_data([None, submission_info], 'test')	
def prep_data(input, split):
	info = input[0]
	data = input[1]
	
	if split=='cv':
		#artists = info.artist.unique()
		artists = info.artist
		#print artists
		#print 'hi', artists
		np.random.shuffle(artists)
		
		info = get_image_info(info, 'train')
		info['bytes_per_pixel'] = 1.0*info['size_bytes']/(info['pixelsx']*info['pixelsy'])
		info['aspect_ratio'] = 1.0*info['pixelsx']/info['pixelsy']
		#train_artists = artists[0:int(0.8*len(artists))]
		#test_artists = artists[int(0.8*len(artists)):]
		#print artists
		print 'hi',info[info.artist.isin(artists)].shape
		train = make_pairs(info[info.artist.isin(artists)])
		#test = make_pairs(info[info.artist.isin(test_artists)])
		#print train.shape
		train['in_train'] = True
		#test['in_train'] = True
		data = train
		data['sameArtist'] = data['artist1'] == data['artist2']
		
	if split=='test':

		info = get_image_info(data, 'test')
		info['bytes_per_pixel'] = 1.0*info['size_bytes']/(info['pixelsx']*info['pixelsy'])
		info['aspect_ratio'] = 1.0*info['pixelsx']/info['pixelsy']	
		
		data['in_train'] = False
	
		if 'artist1' in data.columns:
			data['sameArtist'] = data['artist1'] == data['artist2']

	
	data2 = pd.merge(data, info[['new_filename', 'pixelsx', 'pixelsy', 'size_bytes', 'bytes_per_pixel', 'aspect_ratio']], how='left', left_on='image1', right_on='new_filename')
	data2.drop('new_filename', 1, inplace=True)
	
	data2 = pd.merge(data2, info[['new_filename', 'pixelsx', 'pixelsy', 'size_bytes', 'bytes_per_pixel', 'aspect_ratio']], how='left', left_on='image2', right_on='new_filename')
	data2.drop('new_filename', 1, inplace=True)
	
	x_train = data2[data2.in_train==True][['pixelsx_x', 'pixelsy_x', 'size_bytes_x', 'bytes_per_pixel_x', 'aspect_ratio_x', 'pixelsx_y', 'pixelsy_y', 'size_bytes_y', 'bytes_per_pixel_y', 'aspect_ratio_y']].values
	x_test = data2[data2.in_train==False][['pixelsx_x', 'pixelsy_x', 'size_bytes_x', 'bytes_per_pixel_x', 'aspect_ratio_x', 'pixelsx_y', 'pixelsy_y', 'size_bytes_y', 'bytes_per_pixel_y', 'aspect_ratio_y']].values
	
	
	if 'artist1' in data.columns: 
		y_train = data2[data2.in_train==True]['sameArtist'].values
		y_test = data2[data2.in_train==False]['sameArtist'].values
	else:
		y_test = None	
	
	if split=='cv':		
		return x_train, y_train, x_train, y_train  
 	if split=='test':
		return x_test, y_test


In [2]:
import pandas as pd
import numpy as np
import time

start_time = time.time()
train_info = pd.read_csv('C:\\Users\\swaroop\\Downloads\\painters\\ImageSets\\wyeths_medium.csv')
#submission_info = pd.read_csv('submission_info.csv')
print 'prepping training and cv data'
x_train, y_train, x_cv, y_cv = prep_data([train_info, None], 'cv')

print x_train.shape
#np.savetxt('x_train_wyeth.txt', x_train, fmt = '%1.3f' )
#np.savetxt('y_train_wyeth.txt', y_train, fmt = '%1.3f' )

print (time.time() - start_time)/60 , "minutes"

#print 'prepping test data'
#x_test, y_test = prep_data([None, submission_info], 'test')

prepping training and cv data
 train\Distant Thunder
  train\John Andres House
  train\The Master Bedroom
train\Wind from the Sea
 info shape (50, 9)
hi (50, 11)
make pairs train info shape (50, 11)
a and use shapes (28L, 2L) (22L,)
403.0
use.shape (22L,)
0 0 404 22
a and use shapes (22L, 2L) (28L,)
256.0
use.shape (28L,)
1 426 257 28
(711, 4) (203, 4)
['artist1' 'image1' 'artist2' 'image2']
hi1 (711, 4)
(711L, 10L)
0.0693833311399 minutes


C:\Users\swaroop\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:151: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [14]:
def train_classifier(x_train, y_train, x_cv, y_cv):    
    clf1 = SVC(kernel = 'sigmoid', class_weight  = 'balanced', probability=True, tol = 0.01)
    clf2 = SVC(kernel = 'rbf', class_weight  = 'balanced', probability=True, tol = 0.01)
    clf3 = RandomForestClassifier(n_estimators=10, n_jobs = -1)
    clf4 = GaussianNB()
    clf5 = BernoulliNB() 
    clf6 = LinearSVC(class_weight = 'balanced', dual = False)
    clf7 = SVC(kernel = 'poly', class_weight  = 'balanced', probability=True, tol = 0.1, degree=2, C=1.0)
    clf8 = neighbors.KNeighborsClassifier()
    clf9 = linear_model.LogisticRegression()
    clf10 = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05) #objective='multi:softprob'
    print 'starting fit'
    
    print x_train.shape, y_train.shape
    training, testing, y_training, y_testing = train_test_split(x_train, y_train, test_size=0.3)
    
    print training.shape, testing.shape
    #print y_training, y_testing
    print itemfreq(y_training)
    print itemfreq(y_testing)
    
    clf1.fit(training, y_training)
    clf2.fit(training, y_training)
    clf3.fit(training, y_training)
    clf4.fit(training, y_training)
    clf5.fit(training, y_training)
    clf6.fit(training, y_training)
#     clf7.fit(training, y_training) 
    clf8.fit(training, y_training) 
    clf9.fit(training, y_training)
    clf10.fit(training, y_training)

    print 'SVM - Sigmoid Kernel'
    print_results(clf1, y_testing, clf1.predict(testing), clf1.predict_proba(testing)[:,1] )
    print 'SVM - rbf Kernel'
    print_results(clf2, y_testing, clf2.predict(testing), clf2.predict_proba(testing)[:,1] )
    print 'Random Forest'
    print_results(clf3, y_testing, clf3.predict(testing), clf3.predict_proba(testing)[:,1] )
    print 'Gaussian NB'
    print_results(clf4, y_testing, clf4.predict(testing), clf4.predict_proba(testing)[:,1] )
    print 'Bernoulli NB'
    print_results(clf5, y_testing, clf5.predict(testing), clf5.predict_proba(testing)[:,1] )
    print 'SVM Linear Kernel'    
    prob_pos = clf6.decision_function(testing)
    prob_pos = (prob_pos - prob_pos.min()) / (prob_pos.max() - prob_pos.min())
    print_results(clf6, y_testing, clf6.predict(testing), prob_pos )
#     print 'SVM Polynomial Kernel'
#     print_results(clf7, y_testing, clf7.predict(testing), clf7.predict_proba(testing)[:,1] )
    print 'Nearest Neighbors'
    print_results(clf8, y_testing, clf8.predict(testing), clf8.predict_proba(testing)[:,1] )
    print 'Logistic Regression'
    print_results(clf9, y_testing, clf9.predict(testing), clf9.predict_proba(testing)[:,1] )
    print 'XG Boost'
    print_results(clf10, y_testing, clf10.predict(testing), clf10.predict_proba(testing)[:,1] )


def print_results(clf, y_test, y_pred, y_pred_prob):
    #y_pred_prob = clf.predict_proba(y_test)[:,1]
    #y_pred = clf.predict(y_test)
    print 'ROC - ',roc_auc_score(y_test, y_pred_prob)
    print 'Confusion Matrix - ', confusion_matrix(y_test, y_pred)
    #print 'Precision - ',precision_score(y_test, y_pred ),'Recall - ',recall_score(y_test, y_pred),'F1- Score',f1_score(y_test, y_pred),'\n'
    target_names = ['class 0', 'class 1']
    print classification_report(y_test, y_pred, target_names=target_names)

In [15]:
train_classifier(x_train, y_train, x_train, y_train) 

starting fit
(711L, 10L) (711L,)
(497L, 10L) (214L, 10L)
[[  0  36]
 [  1 461]]
[[  0  14]
 [  1 200]]
SVM - Sigmoid Kernel
ROC -  0.5
Confusion Matrix -  [[  0  14]
 [  0 200]]
             precision    recall  f1-score   support

    class 0       0.00      0.00      0.00        14
    class 1       0.93      1.00      0.97       200

avg / total       0.87      0.93      0.90       214

SVM - rbf Kernel
ROC -  0.5
Confusion Matrix -  [[  0  14]
 [  0 200]]
             precision    recall  f1-score   support

    class 0       0.00      0.00      0.00        14
    class 1       0.93      1.00      0.97       200

avg / total       0.87      0.93      0.90       214

Random Forest
ROC -  0.539285714286
Confusion Matrix -  [[  0  14]
 [  4 196]]
             precision    recall  f1-score   support

    class 0       0.00      0.00      0.00        14
    class 1       0.93      0.98      0.96       200

avg / total       0.87      0.92      0.89       214

Gaussian NB
ROC -  0.482857